In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
pd.set_option('display.max_columns', None)

# 1. Data Preparation

In [3]:
northwind = pd.read_csv('OrdersAgg.csv')
northwind['OrderDate'] = pd.to_datetime(northwind['OrderDate'])
northwind['OrderID'] = northwind['OrderID'].astype('category')
northwind.head()

,OrderID,OrderDate,Recency,Freight,ShippedStatus,LateStatus,ShipCity,ShipCountry,ShipCompanyName,CustCompanyName,TotalQty,TotalSales,GrossProfit,FreightMarkup
0,10248,1996-04-07,672,32.38,1,0,Reims,France,Federal Shipping,Vins et alcools Chevalier,27,440.00,407.62,12.59
1,10249,1996-05-07,671,11.61,1,0,Münster,Germany,Speedy Express,Toms Spezialitäten,49,1863.40,1851.79,159.50
2,10250,1996-08-07,668,65.83,1,0,Rio de Janeiro,Brazil,United Package,Hanari Carnes,60,1552.60,1486.77,22.58
3,10251,1996-08-07,668,41.34,1,0,Lyon,France,Speedy Express,Victuailles en stock,41,654.06,612.72,14.82
4,10252,1996-09-07,667,51.30,1,0,Charleroi,Belgium,United Package,Suprêmes délices,105,3597.90,3546.60,69.13


In [4]:
northwind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   OrderID          830 non-null    category      
 1   OrderDate        830 non-null    datetime64[ns]
 2   Recency          830 non-null    int64         
 3   Freight          830 non-null    float64       
 4   ShippedStatus    830 non-null    int64         
 5   LateStatus       830 non-null    int64         
 6   ShipCity         830 non-null    object        
 7   ShipCountry      830 non-null    object        
 8   ShipCompanyName  830 non-null    object        
 9   CustCompanyName  830 non-null    object        
 10  TotalQty         830 non-null    int64         
 11  TotalSales       830 non-null    float64       
 12  GrossProfit      830 non-null    float64       
 13  FreightMarkup    830 non-null    float64       
dtypes: category(1), datetime64[ns](1), float64

In [5]:
northwind.describe()

,Recency,Freight,ShippedStatus,LateStatus,TotalQty,TotalSales,GrossProfit,FreightMarkup
count,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000
mean,271.657831,78.244205,0.974699,0.044578,61.827711,1525.052024,1446.807819,107.937530
std,197.522339,116.779294,0.157133,0.206501,50.748158,1845.178648,1758.299558,734.877027
min,1.000000,0.020000,0.000000,0.000000,1.000000,12.500000,11.400000,0.100000
25%,92.000000,13.380000,1.000000,0.000000,26.000000,456.692500,432.365000,12.480000
50%,239.500000,41.360000,1.000000,0.000000,50.000000,943.250000,891.935000,19.235000
75%,436.750000,91.430000,1.000000,0.000000,81.000000,1882.200000,1780.787500,39.100000
max,672.000000,1007.640000,1.000000,1.000000,346.000000,16387.500000,16039.360000,12574.000000


# **2. RFM Modelling**

## RFM Metrics Calculation

In [6]:
# Calculate RFM metrics
rfm_nw = northwind.groupby(['CustCompanyName']).agg({
                                        'Recency':'max',
                                        'OrderID':'count',
                                        'TotalSales': 'sum'})
#Rename columns
rfm_nw.rename(columns={'OrderID':'Frequency','TotalSales':'MonetaryValue'}
           ,inplace= True)
#Final RFM values
rfm_nw.head()

,Recency,Frequency,MonetaryValue
CustCompanyName,,,
Alfreds Futterkiste,255,6,4273.00
Ana Trujillo Emparedados y helados,596,4,1402.95
Antonio Moreno Taquería,526,7,7023.98
Around the Horn,538,13,13390.65
B's Beverages,619,10,6089.90


## RFM Segment Calculation

In [8]:
#Building RFM segments
r_labels = np.arange(4,0,-1) # Inverse because smaller is better
f_labels = np.arange(1,5)
m_labels = np.arange(1,5)

#Classification using quartiles, output is as CATEGORY
r_quartiles = pd.qcut(rfm_nw['Recency'], q=4, labels = r_labels)
f_quartiles = pd.qcut(rfm_nw['Frequency'],q=4, labels = f_labels)
m_quartiles = pd.qcut(rfm_nw['MonetaryValue'],q=4, labels = m_labels)
# Adding additional columns
rfm_nw = rfm_nw.assign(R=r_quartiles,F=f_quartiles,M=m_quartiles)

# Build RFM Segment and RFM Score
def add_rfm(x): 
    return str(x['R']) + str(x['F']) + str(x['M'])

rfm_nw['RFM_Segment'] = rfm_nw.apply(add_rfm,axis=1 )
rfm_nw['RFM_Score']   = rfm_nw[['R','F','M']].sum(axis=1)

rfm_nw.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustCompanyName,,,,,,,,
Alfreds Futterkiste,255,6,4273.00,4,2,2,422,8
Ana Trujillo Emparedados y helados,596,4,1402.95,2,1,1,211,4
Antonio Moreno Taquería,526,7,7023.98,3,2,2,322,7
Around the Horn,538,13,13390.65,3,4,3,343,10
B's Beverages,619,10,6089.90,2,3,2,232,7


In [9]:
#Select bottom RFM segment "111" and view top 5 rows
rfm_nw[rfm_nw['RFM_Segment']=='111'].head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustCompanyName,,,,,,,,
Centro comercial Moctezuma,658,1,100.8,1,1,1,111,3
GROSELLA-Restaurante,646,2,1488.7,1,1,1,111,3
Vins et alcools Chevalier,672,5,1480.0,1,1,1,111,3


# **3. Analyze RFM**

## Segment Labelling

In [10]:
# Regex for labelling
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2]3': 'At Risk',
    r'[1-2]4': 'Can\'t Lose',
    r'31': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'32': 'New Customers',
    r'[4-5]2': 'Potential Loyalists',
    r'43': 'Champions'
}

In [11]:
rfm_nw['Segment'] = rfm_nw['R'].astype(str) + rfm_nw['F'].astype(str)
rfm_nw['SegmentCheck'] = rfm_nw['Segment']
rfm_nw['Segment'] = rfm_nw['Segment'].replace(seg_map, regex=True)
rfm_nw.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,Segment,SegmentCheck
CustCompanyName,,,,,,,,,,
Alfreds Futterkiste,255,6,4273.00,4,2,2,422,8,Potential Loyalists,42
Ana Trujillo Emparedados y helados,596,4,1402.95,2,1,1,211,4,Hibernating,21
Antonio Moreno Taquería,526,7,7023.98,3,2,2,322,7,New Customers,32
Around the Horn,538,13,13390.65,3,4,3,343,10,Loyal Customers,34
B's Beverages,619,10,6089.90,2,3,2,232,7,At Risk,23


In [12]:
rfm_nw = rfm_nw.reset_index()
rfm_nw

,CustCompanyName,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,Segment,SegmentCheck
0,Alfreds Futterkiste,255,6,4273.00,4,2,2,422,8,Potential Loyalists,42
1,Ana Trujillo Emparedados y helados,596,4,1402.95,2,1,1,211,4,Hibernating,21
2,Antonio Moreno Taquería,526,7,7023.98,3,2,2,322,7,New Customers,32
3,Around the Horn,538,13,13390.65,3,4,3,343,10,Loyal Customers,34
4,B's Beverages,619,10,6089.90,2,3,2,232,7,At Risk,23
...,...,...,...,...,...,...,...,...,...,...,...
84,Wartian Herkku,650,15,15648.70,1,4,3,143,8,Can't Loose,14
85,Wellington Importadora,661,9,6068.20,1,3,2,132,6,At Risk,13
86,White Clover Markets,645,14,27363.61,1,4,4,144,9,Can't Loose,14
87,Wilman Kala,281,7,3161.35,4,2,1,421,7,Potential Loyalists,42


In [13]:
# rfm_nw.to_csv('rfmResult.csv')